# Avaliando o modelo Sentence-BERT com Cosine Similarity

Neste notebook, vamos avaliar o modelo treinado anteriormente, calculando a similaridade entre pares de sentenças do conjunto de teste e comparando com o **ground truth** (dataset "gabarito").

## 1. Instalação e importação

In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sentence_transformers import SentenceTransformer, util

## 2. Carregar o modelo e os dados

In [29]:
MODEL_PATH_COSINE  = '../output/sbert-cosine'
MODEL_PATH_TRIPLET = '../output/sbert-triplet'
TEST_PATH  = '../data/test.csv'
GT_PATH    = '../data/ground_truth.csv'
SUB_PATH   = '../data/submission.csv'

model = SentenceTransformer(MODEL_PATH_TRIPLET)
df_test  = pd.read_csv(TEST_PATH)
df_gt    = pd.read_csv(GT_PATH)

print(f"🔹 Total de pares de teste: {len(df_test)}")
df_test.head()

🔹 Total de pares de teste: 1517


,id,question,answer
0,0,What do practitioners of Wicca worship ?,"An estimated <num> Americans practice Wicca , ..."
1,1,What do practitioners of Wicca worship ?,The inch- thick chaplain handbook includes a f...
2,2,What do practitioners of Wicca worship ?,Wicca -- sometimes spelled Wycca -- comes from...
3,3,What do practitioners of Wicca worship ?,"Ms . Siefferly , a senior at the high school ,..."
4,4,What do practitioners of Wicca worship ?,"That 's because Ms . Palmer is a witch , the h..."


## 3. Calcular a similaridade entre embeddings

Aqui o modelo transforma cada sentença em um vetor numérico **(embedding)** e calcula o quão parecidos esses vetores são usando a **Cosine Similarity**, gerando a previsão final de similaridade.

In [30]:
emb_q = model.encode(df_test['question'].tolist(), convert_to_tensor=True)
emb_a = model.encode(df_test['answer'].tolist(), convert_to_tensor=True)

# Aplica a Cosine similarity
cos_sim = util.cos_sim(emb_q, emb_a).diagonal().cpu().numpy()

# Normaliza a similaridade do cosseno para o intervalo [0, 1], evitando valores negativos
pred_test = (cos_sim + 1) / 2

df_test['predictions'] = pred_test

## 4. Salvar previsões

In [31]:
# df_test[['id', 'predictions']].to_csv('../data/submission_cosine.csv', index=False)
# print('✅ Previsões salvas em: ../data/submission_cosine.csv')

df_test[['id', 'predictions']].to_csv('../data/submission_triplet.csv', index=False)
print('✅ Previsões salvas em: ../data/submission_triplet.csv')

✅ Previsões salvas em: ../data/submission_triplet.csv


## 5. Avaliar com o Ground Truth

"Ground Truth" é um dataset com as predições corretas, semelhante a um "gabarito" do conjunto de teste.

In [35]:
y_true = pd.to_numeric(df_gt['predictions'], errors='coerce').astype(float).to_numpy()
y_pred = pd.to_numeric(df_test['predictions'], errors='coerce').astype(float).to_numpy()

# y_pred_binary = (y_pred > 0.7).astype(int) # Usando threshold de 0.7 para Cosine similarity
y_pred_binary = (y_pred > 0.8).astype(int)  # Usando threshold de 0.8 para Triplet Loss
y_true_binary = (y_true > 0.5).astype(int)

acc = accuracy_score(y_true_binary, y_pred_binary)
auc = roc_auc_score(y_true_binary, y_pred)
f1  = f1_score(y_true_binary, y_pred_binary)

print(f"✅ Accuracy: {acc:.3f}")
print(f"🔹 AUC: {auc:.3f}")
print(f"🔹 F1-score: {f1:.3f}")

if (auc >= 0.90 and f1 >= 0.70) or (acc >= 0.90 and f1 >= 0.65):
    print("✅ Excelente — modelo bem calibrado e separando muito bem as classes.")
elif (auc >= 0.85 and f1 >= 0.55) or (acc >= 0.85 and f1 >= 0.55):
    print("🟡 Bom — resultado sólido, com boa separação entre as classes.")
elif (auc >= 0.75 and f1 >= 0.45):
    print("⚠️ Razoável — o modelo aprende, mas precisa de refinamento.")
else:
    print("❌ Fraco — reveja os parâmetros de treinamento ou o threshold.")

✅ Accuracy: 0.779
🔹 AUC: 0.814
🔹 F1-score: 0.509
⚠️ Razoável — o modelo aprende, mas precisa de refinamento.


## Conclusão

- O modelo **Sentence-BERT (sBERT)** foi treinado utilizando duas abordagens distintas: **Cosine Similarity Loss** e **Triplet Loss**. Em ambos os casos, o objetivo foi aprender representações vetoriais (embeddings) capazes de capturar a proximidade semântica entre pares de sentenças.
- A avaliação foi feita por meio da **similaridade do cosseno**, que mede o grau de alinhamento entre os embeddings das perguntas e respostas.
- Embora ambos os modelos tenham apresentado resultados consistentes, os experimentos mostraram que há espaço para ajustes de hiperparâmetros e, especialmente no caso do **Triplet Loss**, a exploração de outras funções de perda pode levar a uma melhor generalização, como `MultipleNegativesRankingLoss` ou `ContrastiveLoss`.
- Esse tipo de abordagem pode ser aplicado em diversas tarefas práticas, como:
    - Busca semântica e recuperação de informações por similaridade textual;
    - Detecção de plágio, redundância ou duplicatas em documentos;
    - Sistemas de recomendação de perguntas, respostas ou artigos semelhantes;
    - Análise de coerência e consistência textual em resumos, redações ou contratos.